In [1]:
import numpy as np
import pandas as pd
from math import ceil
from hierarchicalforecast.utils import aggregate
import pickle

c:\py\Anaconda3\lib\site-packages\numba\np\ufunc\parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12040. The TBB threading layer is disabled.
  warnings.warn(problem)


In [2]:

DATA_SOURCE_DIR = "../Data/Original"
PREPROCESSED_DATA_DIR = "../Data/Preprocessed"

# LEVELS_10_5_4_1_DATA_FILE = f"{PREPROCESSED_DATA_DIR}/lvls_10_5_4_1.pkl"
# LEVELS_10_5_4_1_SUMMING_MATRIX = f"{PREPROCESSED_DATA_DIR}/S_lvls_10_5_4_1.pkl"
# LEVELS_10_5_4_1_TAGS = f"{PREPROCESSED_DATA_DIR}/tags_lvls_10_5_4_1.bin"
# LEVELS_9_TO_1_DATA_FILE = f"{PREPROCESSED_DATA_DIR}/lvls_9_to_1.pkl"
# LEVELS_9_TO_1_SUMMING_MATRIX = f"{PREPROCESSED_DATA_DIR}/S_lvls_9_to_1.pkl"
# LEVELS_9_TO_1_TAGS = f"{PREPROCESSED_DATA_DIR}/tags_lvls_9_to_1.bin"
# LEVELS_7_6_2_DATA_FILE = f"{PREPROCESSED_DATA_DIR}/lvls_7_6_2.pkl"
# LEVELS_7_6_2_SUMMING_MATRIX = f"{PREPROCESSED_DATA_DIR}/S_lvls_7_6_2.pkl"
LEVELS_1_TO_12_DATA_FILE = f"{PREPROCESSED_DATA_DIR}/lvls_1_to_12.pkl"
LEVELS_1_TO_12_PRICE_FEATURES = f"{PREPROCESSED_DATA_DIR}/lvls_1_to_12_price_features.pkl"
LEVELS_1_TO_12_SUMMING_MATRIX = f"{PREPROCESSED_DATA_DIR}/S_lvls_1_to_12.pkl"
LEVELS_1_TO_12_TAGS = f"{PREPROCESSED_DATA_DIR}/tags_lvls_1_to_12.bin"
LEVELS_1_AND_12_DATA_FILE = f"{PREPROCESSED_DATA_DIR}/lvls_1_and_12.pkl"
LEVELS_1_AND_12_SUMMING_MATRIX = f"{PREPROCESSED_DATA_DIR}/S_lvls_1_and_12.pkl"
LEVELS_1_AND_12_TAGS = f"{PREPROCESSED_DATA_DIR}/tags_lvls_1_and_12.bin"

STATES = ["CA", "TX", "WI"]
STORES = ["CA_1", "CA_2", "CA_3", "CA_4", "TX_1", "TX_2", "TX_3", "WI_1", "WI_2", "WI_3"]
CATEGORIES = ["FOODS", "HOBBIES", "HOUSEHOLD"]
DEPARTMENTS = ["FOODS_1", "FOODS_2", "FOODS_3", "HOBBIES_1", "HOBBIES_2", "HOUSEHOLD_1", "HOUSEHOLD_2"]

TARGET = "units_sold"
SPEC_LVLS_1_AND_12 = [
    ["total"],  # Level 1,
    ["total", "state_id", "store_id", "cat_id", "dept_id", "item_id"]   # Most disaggregated level (lvl 12)
]
SPEC_LVLS_1_TO_12 = [
    ["total"],  # Level 1
    ["total", "state_id"],  # Level 2
    ["total", "state_id", "store_id"],  # Level 3
    ["total", "cat_id"],    # Level 4
    ["total", "cat_id", "dept_id"], # Level 5
    ["total", "state_id", "cat_id"],    # Level 6
    ["total", "state_id", "cat_id", "dept_id"],   # Level 7
    ["total", "state_id", "store_id", "cat_id"],    # Level 8
    ["total", "state_id", "store_id", "cat_id", "dept_id"],   # Level 9
    ["total", "cat_id", "dept_id", "item_id"],    # Level 10
    ["total", "state_id", "cat_id", "dept_id", "item_id"],   # Level 11
    ["total", "state_id", "store_id", "cat_id", "dept_id", "item_id"]   # Most disaggregated level (lvl 12)
]

In [3]:
def get_level(unique_id: str):
    ids = unique_id.split("/")
    level = 1
    
    if len(ids) == 2:
        if ids[-1] in STATES:
            level = 2
        elif ids[-1] in CATEGORIES:
            level = 4
    elif len(ids) == 3:
        if ids[1] in STATES and ids[2] in STORES:
            level = 3
        elif ids[1] in CATEGORIES and ids[2] in DEPARTMENTS:
            level = 5
        elif ids[1] in STATES and ids[2] in CATEGORIES:
            level = 6
    elif len(ids) == 4:
        if ids[1] in STATES and ids[2] in CATEGORIES and ids[3] in DEPARTMENTS:
            level = 7
        elif ids[1] in STATES and ids[2] in STORES and ids[3] in CATEGORIES:
            level = 8
        else:
            level = 10
    elif len(ids) == 5:
        if ids[2] in STORES:
            level = 9
        else:
            level = 11
    elif len(ids) == 6:
        level = 12

    return level


def get_hierarchical_ts_df(spec: list, base_df: pd.DataFrame):
    # Get hierarchical demand data
    hierarchical_cols = [
        "ds", 
        "y", 
        "total",
        'state_id',
        'store_id',
        'cat_id',
        'dept_id',
        'item_id'
    ]
    Y_df, S_df, tags = aggregate(base_df[hierarchical_cols], spec)
    Y_df.reset_index(inplace=True)
    print(Y_df)

    # Get date features
    date_feature_cols = [
        'ds',
        'tm_d',
        'tm_w',
        'tm_m',
        'tm_y',
        'tm_wm',
        'tm_dw',
        'tm_w_end'
    ]
    dates = base_df[date_feature_cols].drop_duplicates().set_index('ds')
    Y_df = Y_df.set_index("ds").join(dates, how="left").reset_index()
    del dates

    # Get event features
    event_feature_cols = [
        'ds',
        'event_name_1',
        'event_type_1',
        'event_name_2',
        'event_type_2',
        'snap_CA',
        'snap_TX',
        'snap_WI'
    ]
    events = base_df[event_feature_cols].drop_duplicates().set_index('ds')
    Y_df = Y_df.set_index("ds").join(events, how="left").reset_index()
    del events

    # Get level
    Y_df["level"] = Y_df["unique_id"].apply(get_level)

    return Y_df, S_df, tags


def get_hierarchical_price_features(spec: list, base_df: pd.DataFrame):
    # Get aggregate price features
    bottom_lvl_prices_cols = [
        "ds", 
        "sell_price", 
        "total",
        'state_id',
        'store_id',
        'cat_id',
        'dept_id',
        'item_id'
    ]
    bottom_lvl_prices = base_df[bottom_lvl_prices_cols]
    bottom_lvl_prices.rename(columns={"sell_price": "y"}, inplace=True)
    prices_df, _, _ = aggregate(bottom_lvl_prices, spec)
    prices_df.rename(columns={"y": "sell_price"}, inplace=True)
    del bottom_lvl_prices
    return prices_df.reset_index()

### Base Dataframe

In [4]:
# Load csvs from Kaggle - https://www.kaggle.com/c/m5-forecasting-accuracy/data
sales_df = pd.read_csv(f"sales_train_evaluation.csv")
prices_df = pd.read_csv(f"sell_prices.csv")
calendar_df = pd.read_csv(f"calendar.csv")

# Convert to long format dataframe
index_cols = [
    "id",
    "item_id",
    "dept_id",
    "cat_id",
    "store_id",
    "state_id"
]
grid_df = pd.melt(
    sales_df,
    id_vars=index_cols,
    var_name="d",
    value_name=TARGET
)
del sales_df

# We can free some memory by converting "strings" to categorical. It will not affect merging and we will not lose any valuable data
for col in index_cols:
    grid_df[col] = grid_df[col].astype('category')

print(grid_df["d"].unique())
print(grid_df.info())

['d_1' 'd_2' 'd_3' ... 'd_1939' 'd_1940' 'd_1941']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59181090 entries, 0 to 59181089
Data columns (total 8 columns):
 #   Column      Dtype   
---  ------      -----   
 0   id          category
 1   item_id     category
 2   dept_id     category
 3   cat_id      category
 4   store_id    category
 5   state_id    category
 6   d           object  
 7   units_sold  int64   
dtypes: category(6), int64(1), object(1)
memory usage: 1.3+ GB
None


### Calendar Features

In [5]:
%%time
calendar_index_cols = [
    'date',
    'wm_yr_wk',
    'd',
    'event_name_1',
    'event_type_1',
    'event_name_2',
    'event_type_2',
    'snap_CA',
    'snap_TX',
    'snap_WI'
]
grid_df = grid_df.merge(calendar_df[calendar_index_cols], on=["d"], how="left")
del calendar_df
for col in calendar_index_cols[2:]:
    grid_df[col] = grid_df[col].astype("category")

# Convert date column to DateTime
grid_df["date"] = pd.to_datetime(grid_df["date"])
print(grid_df.head())

# DateTime feature engineering
grid_df['tm_d'] = grid_df['date'].dt.day.astype(np.int8)
grid_df['tm_w'] = grid_df['date'].dt.isocalendar().week.astype(np.int8)
grid_df['tm_m'] = grid_df['date'].dt.month.astype(np.int8)
grid_df['tm_y'] = grid_df['date'].dt.year
grid_df['tm_y'] = (grid_df['tm_y'] - grid_df['tm_y'].min()).astype(np.int8)
grid_df['tm_wm'] = grid_df['tm_d'].apply(lambda x: ceil(x/7)).astype(np.int8)
grid_df['tm_dw'] = grid_df['date'].dt.dayofweek.astype(np.int8) 
grid_df['tm_w_end'] = (grid_df['tm_dw']>=5).astype(np.int8)

print(grid_df.columns)
print(grid_df.info())
print(grid_df.head())

                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id    d  units_sold       date  wm_yr_wk event_name_1 event_type_1  \
0       CA  d_1           0 2011-01-29     11101          NaN          NaN   
1       CA  d_1           0 2011-01-29     11101          NaN          NaN   
2       CA  d_1           0 2011-01-29     11101          NaN          NaN   
3       CA  d_1           0 2011-01-29     11101          NaN          NaN   
4       CA  d_1           0 2011-01-29     11101          NaN          NaN   

  event_name_2 event_type_2 snap_CA snap_TX snap_W

### Price Features

In [6]:
%%time
price_index_cols = [
    "store_id",
    "item_id",
    "wm_yr_wk"
]
grid_df = grid_df.merge(prices_df, on=price_index_cols, how="left")
for col in price_index_cols[:2]:
    grid_df[col] = grid_df[col].astype("category")
grid_df = grid_df.drop(columns=["wm_yr_wk"])
del prices_df
grid_df["sell_price"] = grid_df["sell_price"].fillna(0) #  Replace trailing NaN prices with 0.

print(grid_df.shape)
print(grid_df.info())

(59181090, 24)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 59181090 entries, 0 to 59181089
Data columns (total 24 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            category      
 1   item_id       category      
 2   dept_id       category      
 3   cat_id        category      
 4   store_id      category      
 5   state_id      category      
 6   d             category      
 7   units_sold    int64         
 8   date          datetime64[ns]
 9   event_name_1  category      
 10  event_type_1  category      
 11  event_name_2  category      
 12  event_type_2  category      
 13  snap_CA       category      
 14  snap_TX       category      
 15  snap_WI       category      
 16  tm_d          int8          
 17  tm_w          int8          
 18  tm_m          int8          
 19  tm_y          int8          
 20  tm_wm         int8          
 21  tm_dw         int8          
 22  tm_w_end      int8          
 23  sell_price    f

In [7]:
%%time
# Formatting for Nixtla aggregate function
grid_df.rename(columns={TARGET: "y", "d": "ds"}, inplace=True)
grid_df["total"] = "TOTAL"  # Dummy column for highest aggregation level.
grid_df["total"] = grid_df["total"].astype("category")
grid_df["ds"] = grid_df['ds'].apply(lambda x: x[2:])
grid_df["ds"] = grid_df["ds"].astype(np.int16)
grid_df[["ds", "y", "total"] + index_cols].head()

Wall time: 3.86 s


,ds,y,total,id,item_id,dept_id,cat_id,store_id,state_id
0,1,0,TOTAL,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA
1,1,0,TOTAL,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA
2,1,0,TOTAL,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA
3,1,0,TOTAL,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA
4,1,0,TOTAL,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA


### Summing matrix and tags for levels 1 and 12

In [8]:
# S_df_lvls_1_and_12.to_pickle(LEVELS_1_AND_12_SUMMING_MATRIX)
# with open(LEVELS_1_AND_12_TAGS, "wb") as f:
#     pickle.dump(tags_lvls_1_and_12, f)

### Hierarchical Time Series Levels 1 - 12

In [9]:
Y_df_lvls_1_to_12, S_df_lvls_1_to_12, tags_lvls_1_to_12 = get_hierarchical_ts_df(SPEC_LVLS_1_TO_12, grid_df)
Y_df_lvls_1_to_12

             index                                          unique_id    ds  \
0                0                                              TOTAL     1   
1                1                                              TOTAL     2   
2                2                                              TOTAL     3   
3                3                                              TOTAL     4   
4                4                                              TOTAL     5   
...            ...                                                ...   ...   
83152435  83152435  TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...  1937   
83152436  83152436  TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...  1938   
83152437  83152437  TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...  1939   
83152438  83152438  TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...  1940   
83152439  83152439  TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...  1941   

              y  
0         32631  
1         31749

,ds,index,unique_id,y,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,level
0,1,0,TOTAL,32631,29,4,1,0,5,5,1,NaN,NaN,NaN,NaN,0,0,0,1
1,1,1941,TOTAL/CA,14195,29,4,1,0,5,5,1,NaN,NaN,NaN,NaN,0,0,0,2
2,1,3882,TOTAL/TX,9438,29,4,1,0,5,5,1,NaN,NaN,NaN,NaN,0,0,0,2
3,1,5823,TOTAL/WI,8998,29,4,1,0,5,5,1,NaN,NaN,NaN,NaN,0,0,0,2
4,1,7764,TOTAL/FOODS,23178,29,4,1,0,5,5,1,NaN,NaN,NaN,NaN,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83152435,1941,83144675,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,0,22,20,5,5,4,6,1,NaN,NaN,NaN,NaN,0,0,0,12
83152436,1941,83146616,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,0,22,20,5,5,4,6,1,NaN,NaN,NaN,NaN,0,0,0,12
83152437,1941,83148557,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,0,22,20,5,5,4,6,1,NaN,NaN,NaN,NaN,0,0,0,12
83152438,1941,83150498,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,0,22,20,5,5,4,6,1,NaN,NaN,NaN,NaN,0,0,0,12


In [10]:
Y_df_lvls_1_to_12.groupby(by="level")["level"].count()/1941

level
1         1.0
2         3.0
3        10.0
4         3.0
5         7.0
6         9.0
7        21.0
8        30.0
9        70.0
10     3049.0
11     9147.0
12    30490.0
Name: level, dtype: float64

In [13]:
S_df_lvls_1_to_12.to_pickle("S_df.pkl")

# 1

In [11]:
Y_df, S_df, tags = Y_df_lvls_1_to_12, S_df_lvls_1_to_12, tags_lvls_1_to_12
del Y_df_lvls_1_to_12, S_df_lvls_1_to_12, tags_lvls_1_to_12

In [14]:
S_df_lvls_1_to_12

,unique_id,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_001,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_002,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_003,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_004,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_005,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_006,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_008,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_009,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_010,...,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_507,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_508,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_509,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_510,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_511,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_512,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_513,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_514,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_515,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_516
0,TOTAL,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,TOTAL/CA,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,TOTAL/TX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TOTAL/WI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,TOTAL/FOODS,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42835,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
42836,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
42837,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
42838,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
a = S_df["unique_id"]

In [20]:
S_df = S_df.set_index("unique_id")
S_df

,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_001,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_002,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_003,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_004,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_005,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_006,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_008,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_009,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_010,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_011,...,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_507,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_508,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_509,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_510,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_511,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_512,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_513,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_514,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_515,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_516
unique_id,,,,,,,,,,,,,,,,,,,,,
TOTAL,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
TOTAL/CA,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TOTAL/TX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TOTAL/WI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
TOTAL/FOODS,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
from hierarchicalforecast.utils import is_strictly_hierarchical
is_strictly_hierarchical(S_df.T, tags)

In [24]:
S_df.T.columns

Index(['TOTAL', 'TOTAL/CA', 'TOTAL/TX', 'TOTAL/WI', 'TOTAL/FOODS',
       'TOTAL/HOBBIES', 'TOTAL/HOUSEHOLD', 'TOTAL/CA/CA_1', 'TOTAL/CA/CA_2',
       'TOTAL/CA/CA_3',
       ...
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_507',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_508',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_509',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_510',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_511',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_512',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_513',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_514',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_515',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_516'],
      dtype='object', name='unique_id', length=42840)

In [40]:
print(S_df.shape)
print(S_df.columns)
print(S_df.head())
print(is_strictly_hierarchical(S_df, tags))

(42840, 30490)
Index(['TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_001',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_002',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_003',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_004',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_005',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_006',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_008',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_009',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_010',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_011',
       ...
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_507',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_508',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_509',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_510',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_511',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_512',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_513',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOL

KeyError: "None of [Index(['TOTAL'], dtype='object')] are in the [columns]"

In [30]:
for _, idx in tags.items():
    print(idx)
    if not set(idx).issubset(S_df.index):
        print(f"Missing index: {set(idx) - set(S_df.index)}")

['TOTAL']
['TOTAL/CA' 'TOTAL/TX' 'TOTAL/WI']
['TOTAL/FOODS' 'TOTAL/HOBBIES' 'TOTAL/HOUSEHOLD']
['TOTAL/CA/CA_1' 'TOTAL/CA/CA_2' 'TOTAL/CA/CA_3' 'TOTAL/CA/CA_4'
 'TOTAL/TX/TX_1' 'TOTAL/TX/TX_2' 'TOTAL/TX/TX_3' 'TOTAL/WI/WI_1'
 'TOTAL/WI/WI_2' 'TOTAL/WI/WI_3']
['TOTAL/FOODS/FOODS_1' 'TOTAL/FOODS/FOODS_2' 'TOTAL/FOODS/FOODS_3'
 'TOTAL/HOBBIES/HOBBIES_1' 'TOTAL/HOBBIES/HOBBIES_2'
 'TOTAL/HOUSEHOLD/HOUSEHOLD_1' 'TOTAL/HOUSEHOLD/HOUSEHOLD_2']
['TOTAL/CA/FOODS' 'TOTAL/CA/HOBBIES' 'TOTAL/CA/HOUSEHOLD' 'TOTAL/TX/FOODS'
 'TOTAL/TX/HOBBIES' 'TOTAL/TX/HOUSEHOLD' 'TOTAL/WI/FOODS'
 'TOTAL/WI/HOBBIES' 'TOTAL/WI/HOUSEHOLD']
['TOTAL/CA/FOODS/FOODS_1' 'TOTAL/CA/FOODS/FOODS_2'
 'TOTAL/CA/FOODS/FOODS_3' 'TOTAL/CA/HOBBIES/HOBBIES_1'
 'TOTAL/CA/HOBBIES/HOBBIES_2' 'TOTAL/CA/HOUSEHOLD/HOUSEHOLD_1'
 'TOTAL/CA/HOUSEHOLD/HOUSEHOLD_2' 'TOTAL/TX/FOODS/FOODS_1'
 'TOTAL/TX/FOODS/FOODS_2' 'TOTAL/TX/FOODS/FOODS_3'
 'TOTAL/TX/HOBBIES/HOBBIES_1' 'TOTAL/TX/HOBBIES/HOBBIES_2'
 'TOTAL/TX/HOUSEHOLD/HOUSEHOLD_1' 'TOTAL/TX/HO

In [32]:
for _, idx in tags.items():
    if not set(idx).issubset(S_df.index):
        print(f"Missing index: {set(idx) - set(S_df.index)}")

In [33]:
set(['TOTAL']).issubset(S_df.index)

True

In [39]:
tags.items()

dict_items([('total', array(['TOTAL'], dtype=object)), ('total/state_id', array(['TOTAL/CA', 'TOTAL/TX', 'TOTAL/WI'], dtype=object)), ('total/cat_id', array(['TOTAL/FOODS', 'TOTAL/HOBBIES', 'TOTAL/HOUSEHOLD'], dtype=object)), ('total/state_id/store_id', array(['TOTAL/CA/CA_1', 'TOTAL/CA/CA_2', 'TOTAL/CA/CA_3', 'TOTAL/CA/CA_4',
       'TOTAL/TX/TX_1', 'TOTAL/TX/TX_2', 'TOTAL/TX/TX_3', 'TOTAL/WI/WI_1',
       'TOTAL/WI/WI_2', 'TOTAL/WI/WI_3'], dtype=object)), ('total/cat_id/dept_id', array(['TOTAL/FOODS/FOODS_1', 'TOTAL/FOODS/FOODS_2',
       'TOTAL/FOODS/FOODS_3', 'TOTAL/HOBBIES/HOBBIES_1',
       'TOTAL/HOBBIES/HOBBIES_2', 'TOTAL/HOUSEHOLD/HOUSEHOLD_1',
       'TOTAL/HOUSEHOLD/HOUSEHOLD_2'], dtype=object)), ('total/state_id/cat_id', array(['TOTAL/CA/FOODS', 'TOTAL/CA/HOBBIES', 'TOTAL/CA/HOUSEHOLD',
       'TOTAL/TX/FOODS', 'TOTAL/TX/HOBBIES', 'TOTAL/TX/HOUSEHOLD',
       'TOTAL/WI/FOODS', 'TOTAL/WI/HOBBIES', 'TOTAL/WI/HOUSEHOLD'],
      dtype=object)), ('total/state_id/cat_id/dept_id',

In [ ]:
np.argmax(S_df['TOTAL'], axis=0)

In [49]:
S_df.loc['TOTAL',:]

TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_001                1.0
TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_002                1.0
TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_003                1.0
TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_004                1.0
TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_005                1.0
                                                      ... 
TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_512    1.0
TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_513    1.0
TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_514    1.0
TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_515    1.0
TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_516    1.0
Name: TOTAL, Length: 30490, dtype: float64

In [50]:
S_df.index

Index(['TOTAL', 'TOTAL/CA', 'TOTAL/TX', 'TOTAL/WI', 'TOTAL/FOODS',
       'TOTAL/HOBBIES', 'TOTAL/HOUSEHOLD', 'TOTAL/CA/CA_1', 'TOTAL/CA/CA_2',
       'TOTAL/CA/CA_3',
       ...
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_507',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_508',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_509',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_510',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_511',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_512',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_513',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_514',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_515',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_516'],
      dtype='object', name='unique_id', length=42840)

In [51]:
S_df.columns

Index(['TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_001',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_002',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_003',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_004',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_005',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_006',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_008',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_009',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_010',
       'TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_011',
       ...
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_507',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_508',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_509',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_510',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_511',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_512',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_513',
       'TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2

In [52]:
S_df

,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_001,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_002,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_003,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_004,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_005,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_006,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_008,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_009,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_010,TOTAL/CA/CA_1/FOODS/FOODS_1/FOODS_1_011,...,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_507,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_508,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_509,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_510,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_511,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_512,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_513,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_514,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_515,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_516
unique_id,,,,,,,,,,,,,,,,,,,,,
TOTAL,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
TOTAL/CA,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TOTAL/TX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TOTAL/WI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
TOTAL/FOODS,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_2_514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [53]:
tags

{'total': array(['TOTAL'], dtype=object),
 'total/state_id': array(['TOTAL/CA', 'TOTAL/TX', 'TOTAL/WI'], dtype=object),
 'total/cat_id': array(['TOTAL/FOODS', 'TOTAL/HOBBIES', 'TOTAL/HOUSEHOLD'], dtype=object),
 'total/state_id/store_id': array(['TOTAL/CA/CA_1', 'TOTAL/CA/CA_2', 'TOTAL/CA/CA_3', 'TOTAL/CA/CA_4',
        'TOTAL/TX/TX_1', 'TOTAL/TX/TX_2', 'TOTAL/TX/TX_3', 'TOTAL/WI/WI_1',
        'TOTAL/WI/WI_2', 'TOTAL/WI/WI_3'], dtype=object),
 'total/cat_id/dept_id': array(['TOTAL/FOODS/FOODS_1', 'TOTAL/FOODS/FOODS_2',
        'TOTAL/FOODS/FOODS_3', 'TOTAL/HOBBIES/HOBBIES_1',
        'TOTAL/HOBBIES/HOBBIES_2', 'TOTAL/HOUSEHOLD/HOUSEHOLD_1',
        'TOTAL/HOUSEHOLD/HOUSEHOLD_2'], dtype=object),
 'total/state_id/cat_id': array(['TOTAL/CA/FOODS', 'TOTAL/CA/HOBBIES', 'TOTAL/CA/HOUSEHOLD',
        'TOTAL/TX/FOODS', 'TOTAL/TX/HOBBIES', 'TOTAL/TX/HOUSEHOLD',
        'TOTAL/WI/FOODS', 'TOTAL/WI/HOBBIES', 'TOTAL/WI/HOUSEHOLD'],
       dtype=object),
 'total/state_id/cat_id/dept_id': array(['T